# Save whole forcing timeseries of WS and precipitation at locations of selected hurricane events (forcing available from 2011 to 2015) 

### Need to have huge forcing files on computer in Downloads folder

In [3]:
import xarray as xr
import csv
import pandas as pd
import pickle
import numpy as np
import os
import datetime #import datetime, date, timedelta
import sys
import matplotlib.pyplot as plt
import netCDF4 as nc
import os.path
import webbrowser
import time
from scipy.io import netcdf
import netCDF4
from datetime import date, timedelta, datetime
from FUNCTIONS_HYCOM import set_regions, set_regions_tags

In [4]:
years = np.arange(2011,2016) ################################### CHANGEEEEEEE

varname_1 = 'wndspd' # wind
varname_2 = 'precip' # precipitation


# lat_max, lat_min, lon_max, lon_min for every region
regions = set_regions()
regions_tags = set_regions_tags(regions)

savedir = '/Users/jacoposala/Desktop/CU/3.RESEARCH/ARGO_analysis/TC_HYCOM/DATA/FORCING_alltimes_singleevents/'


In [6]:

for r,r_tag in zip(regions, regions_tags):
    lat_min = r[1]
    lat_max = r[0]
    lon_min = r[3]
    lon_max = r[2]
    
    # Open .csv file with selected hurricane events in the chosen region
    df_forcing = pd.read_csv('df_forcing_hur_events_NEW_BOX_with_functions_' + str(lat_min) + '_' + str(lat_max) + '_' + str(lon_min) + '_' + str(lon_max) + '.csv')
    # Convert to [0,360] to match huge forcing convention
    df_forcing.LONG = df_forcing.LONG % 360

    for year in years:
        # Open forcing for this year
        filename_precip='/Users/jacoposala/Downloads/cfsv2-sea_' + str(year) + '_01hr_precip.nc' # precipitation rate
        filename_windspd='/Users/jacoposala/Downloads/cfsv2-sec2_' + str(year) + '_01hr_wndspd.nc'
        ds_precip=xr.open_dataset(filename_precip,decode_times=False)
        ds_windspd=xr.open_dataset(filename_windspd,decode_times=False)
        print(year)
        for i in np.arange(df_forcing.shape[0]):
            # Latitude and longitude of event being considered
            sel_lat = df_forcing.iloc[i].LAT
            sel_lon = df_forcing.iloc[i].LONG
            # Central time of event being considered
            sel_date = df_forcing.iloc[i].DATETIME

            # Extract forcing time series at single location
            sel_ws = ds_windspd[varname_1].where((abs(ds_windspd.Latitude - sel_lat) < 0.01) & (abs(ds_windspd.Longitude - sel_lon) < 0.01), drop=True)
            sel_precipitation = ds_precip[varname_2].where((abs(ds_precip.Latitude - sel_lat) < 0.01) & (abs(ds_precip.Longitude - sel_lon) < 0.01), drop=True)

            # Save it in DataFrame
            column_names = ['lat','lon','time','wind_speed','precipitation']
            df_all = pd.DataFrame(columns = column_names)

            start = datetime(1900,12,31,0,0,0) # This is the "days since" part

            # Populate DataFrame
            for j in np.arange(sel_ws.shape[0]):
                temp = pd.DataFrame(
                        {
                            # Lat
                            'lat': float(sel_ws.Latitude.values),
                            # Lon
                            'lon': float(sel_ws.Longitude.values),
                            # Time
                            'time': [start + timedelta(float(sel_ws[j].MT.values))],
                            # WS
                            'wind_speed': float(sel_ws[j].values),
                            # Precipitation
                            'precipitation': float(sel_precipitation[j].values)
                        })

                df_all = pd.concat([df_all, temp])
            
            # Save
            df_all.to_csv(savedir + 'df_forcing_' + str(sel_lat) + '_' + str(sel_lon) + '_' + str(year) + '_' + r_tag + '.csv')


FileNotFoundError: [Errno 2] No such file or directory: b'/Users/jacoposala/Downloads/cfsv2-sea_2011_01hr_precip.nc'